In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [3]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium['EX_h2_e'] = 1000
medium['EX_h2o_e'] = 1000
medium['EX_k_e'] = 0
medium['EX_fru_e'] = 0
medium['EX_mg2_e'] = 0
medium['EX_mn2_e'] = 0
medium['EX_na1_e'] = 0
medium['EX_nh4_e'] = 0
medium['EX_ni2_e'] = 0
medium['EX_pi_e'] = 0
medium['EX_mobd_e'] = 0
medium['EX_zn2_e'] = 0
medium['EX_ribflv_e'] = 0
medium['EX_btn_e'] = 0
medium['EX_so4_e'] = 0
medium['EX_thm_e'] = 0
medium['EX_ca2_e'] = 0
medium['EX_cl_e'] = 0
medium['EX_co2_e'] = 10
medium['EX_cobalt2_e'] = 0
medium['EX_cu2_e'] = 0
medium['EX_fe2_e'] = 0
medium['EX_fol_e'] = 0
medium['EX_h_e'] = 1000
Cljung_Model.medium = medium

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [4]:
Cljung_Model.reactions.ATPM.bounds = 0, 1000

Cljung_Model.reactions.ACt2r.knock_out()

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})

mthfd2 = Reaction("MTHFD2")
mthfd2.name = "Methylenetetrahydrofolate dehydrogenase (NAD)"

mthfd2.add_metabolites({
    Cljung_Model.metabolites.mlthf_c: -1.0,
    Cljung_Model.metabolites.nad_c: -1.0,
    Cljung_Model.metabolites.methf_c: 1.0,
    Cljung_Model.metabolites.nadh_c: 1.0,
})
print(mthfd2.check_mass_balance())    #ensure correct mass balance
print(mthfd2.bounds)                  #Ensure correct bounds
print(mthfd2.reaction)

Cljung_Model.add_reactions([acT, mthfd2])
Cljung_Model.reactions.MTHFD2.bounds = -1000.0, 1000.0


{}
(0.0, 1000.0)
mlthf_c + nad_c --> methf_c + nadh_c


In [5]:
Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.HYDFDN.bounds = -1000.0, 1000.0

# Find core reactions manually

In [6]:
#Cljung_Model.objective = "ATPM"
#Cljung_Model.objective = "EX_ac_e"
Cljung_Model.objective = "EX_etoh_e"
#Cljung_Model.objective = "EX_but_e"
#Cljung_Model.objective = "EX_btoh_e"
#Cljung_Model.objective = "EX_btd_RR_e"
#print(Cljung_Model.slim_optimize())
#solution = Cljung_Model.optimize()
#Cljung_Model.pfba()
#solution.fluxes[solution.fluxes != 0].index

solution = cobra.flux_analysis.pfba(Cljung_Model)
solution.fluxes[solution.fluxes != 0].index

Index(['EX_h2_e', 'EX_h2o_e', 'EX_co2_e', 'EX_etoh_e', 'ACKr', 'CO2t',
       'ALCD2y', 'ACAFDOR', 'CODH_ACS', 'ETOHt', 'H2Ot', 'H2td', 'FDH7',
       'FRNDPR2r', 'FTHFLi', 'METR', 'MTHFD', 'MTHFC', 'MTHFR5', 'PTAr',
       'HYDFDN'],
      dtype='object')

In [7]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_pi_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD2,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,

Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.RNF,
    
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
    
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

# Add Acetone Pathway

In [8]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [9]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: -1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c + h_c --> acac_c + accoa_c


In [10]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
#    WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c --> act_c + co2_c


In [11]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07f89ae5d1710
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07f89ae5c4a20
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
WLmodelActn.add_reactions([ctfAB_ac, adc, actT, act_ex])

In [14]:
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("atp_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("adp_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("nadh_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("nad_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("fdxo_42_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("fdxr_42_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("coa_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("h_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("pi_c"), type="sink")

Reaction identifier,SK_pi_c
Name,Phosphate sink
Memory address,0x07f89ae54bdd8
Stoichiometry,pi_c <=> Phosphate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Use this instead:
https://www.frontiersin.org/articles/10.3389/fbioe.2020.621166/full

In [15]:
medium = WLmodelActn.medium

medium['EX_co2_e'] = 10
medium['EX_h2_e'] = 1000
medium['EX_co_e'] = 0

WLmodelActn.medium = medium

In [16]:
WLmodelActn.reactions.CODH_ACS.bounds = 1,1

In [17]:
WLmodelActn.objective = "EX_ac_e"
WLmodelActn.objective = "EX_etoh_e"
WLmodelActn.objective = "EX_act_e"
WLmodelActn.objective = "EX_but_e"
WLmodelActn.objective = "EX_btoh_e"
WLmodelActn.objective = "EX_btd_RR_e"

solution = WLmodelActn.optimize()

print("Product: ",WLmodelActn.slim_optimize())
print("ATP: ",WLmodelActn.reactions.SK_atp_c.flux)
print("NADH: ",WLmodelActn.reactions.SK_nadh_c.flux)
print("Ferredoxin: ",WLmodelActn.reactions.SK_fdxo_42_c.flux)
print("H+: ",WLmodelActn.reactions.SK_h_c.flux)

WLmodelActn.optimize()

Product:  0.5
ATP:  -1.0
NADH:  -3.5
Ferredoxin:  2.0
H+:  -6.5


,fluxes,reduced_costs
EX_h2_e,0.0,-0.0
EX_h2o_e,2.0,0.0
EX_co2_e,-2.0,-0.0
EX_co_e,0.0,0.0
EX_pi_e,0.0,0.0
...,...,...
SK_nad_c,3.5,0.0
SK_fdxo_42_c,2.0,0.0
SK_fdxr_42_c,-2.0,-0.0
SK_h_c,-6.5,-0.0


In [18]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn_SK.json'))#, reaction_data = solution.fluxes.to_dict())

Builder()

In [19]:
cobra.io.write_sbml_model(WLmodelActn, join(data_dir, "Models/WLmodelActn.xml"))#, use_fbc_package=False)

## Add synthetic Acetone pathway of LanzaTech
https://info.ornl.gov/sites/publications/Files/Pub127156.pdf

In [20]:
WLmodelActn2 = WLmodel

WLmodelActn2.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn2.add_metabolites(Metabolite(id='aap_c',formula = 'C4H5O6P', name = 'Acetoacetyl-P', compartment = 'c' ))
WLmodelActn2.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn2.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [21]:
ptb = Reaction("Ptb")
ptb.name = "Phosphate butyryl transferase"

ptb.add_metabolites({
    WLmodelActn2.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn2.metabolites.pi_c: -1.0,      #Phosphate
    WLmodelActn2.metabolites.coa_c: 1.0,      #Coenzyme A
    WLmodelActn2.metabolites.aap_c: 1.0,      #Acetoacetyl-P
})
print(ptb.check_mass_balance())    #Ensure correct mass balance
print(ptb.bounds)                  #Ensure correct bounds
print(ptb.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + pi_c --> aap_c + coa_c


In [22]:
buk = Reaction("Buk")
buk.name = "Butyrate kinase"

buk.add_metabolites({
    WLmodelActn2.metabolites.aap_c: -1.0,     #Acetoacetyl-P
    WLmodelActn2.metabolites.h_c: -1.0,       #hydrogen, not mentioned, required for balancing
    WLmodelActn2.metabolites.adp_c: -1.0,     #adp
    WLmodelActn2.metabolites.atp_c: 1.0,      #atp
    WLmodelActn2.metabolites.acac_c: 1.0,     #Acetyl-CoA
})
print(buk.check_mass_balance())    #Ensure correct mass balance
print(buk.bounds)                  #Ensure correct bounds
print(buk.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aap_c + adp_c + h_c --> acac_c + atp_c


In [23]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn2.metabolites.act_c: -1.0,
    WLmodelActn2.metabolites.act_e: 1.0
})

act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn2.metabolites.act_e: -1.0,
})

WLmodelActn2.add_reactions([ptb, buk, actT, act_ex])

Ignoring reaction 'ACTt' since it already exists.
Ignoring reaction 'EX_act_e' since it already exists.


In [24]:
WLmodelActn.objective = "EX_act_e"
WLmodelActn.optimize()
#solution = WLmodel.optimize()
WLmodelActn.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,1.5,1,3.09%
h2_e,EX_h2_e,2,0,0.00%
atp_c,SK_atp_c,0.5,10,10.31%
h_c,SK_h_c,1.5,0,0.00%
nadh_c,SK_nadh_c,2,21,86.60%
Metabolite,Reaction,Flux,C-Number,C-Flux
act_e,EX_act_e,-0.5,3,3.09%
h2o_e,EX_h2o_e,-2,0,0.00%
adp_c,SK_adp_c,-0.5,10,10.31%
nad_c,SK_nad_c,-2,21,86.60%


In [27]:
WLmodelActn2.objective = "EX_act_e"
WLmodelActn2.optimize()
solutionWLActn2 = WLmodel.optimize()
WLmodelActn2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,1.5,1,3.09%
h2_e,EX_h2_e,2,0,0.00%
atp_c,SK_atp_c,0.5,10,10.31%
h_c,SK_h_c,1.5,0,0.00%
nadh_c,SK_nadh_c,2,21,86.60%
Metabolite,Reaction,Flux,C-Number,C-Flux
act_e,EX_act_e,-0.5,3,3.09%
h2o_e,EX_h2o_e,-2,0,0.00%
adp_c,SK_adp_c,-0.5,10,10.31%
nad_c,SK_nad_c,-2,21,86.60%


In [28]:
Builder(model = WLmodelActn2, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn_SK.json'), reaction_data = solutionWLActn2.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': -2.0, 'EX_h2o_e': 2.0, 'EX_co2_e': -1.5, 'EX_co_e': 0.0, 'EX_pi_e': 0.0, 'EX…

The proposed synthetic Acetone pathway is not an improvement compared to the ABE one. Emphasizes the importances of using stoichiometric models.